## Example submission

Image Matching Challenge 2025: https://www.kaggle.com/competitions/image-matching-challenge-2025

This notebook creates a simple submission using ALIKED and LightGlue, plus DINO for shortlisting, on GPU. Adapted from [last year](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

Remember to select an accelerator on the sidebar to the right, and to disable internet access when submitting a notebook to the competition.

In [1]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.8
    Uninstalling kornia_rs-0.1.8:
      Successfully uninstalled kornia_rs-0.1.8
  Attempting uninstall: kornia
    Found existing installation: kornia 0.8.0
    Uninstalling kornia-0.8.0:
      Successfully uninstalled ko

In [2]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [3]:

print("PyTorch version:", torch.__version__)
import sys
print("Python version:", sys.version)

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


PyTorch version: 2.5.1+cu121
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
CUDA available: True
CUDA version: 12.1
Device count: 2
Current device: 0
Device name: Tesla T4


In [4]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cuda', index=0)


In [5]:
def load_pil_image(fname):
    """Loads an image using PIL."""
    return Image.open(fname).convert('RGB')

def get_image_size(fname):
    """Gets image size (width, height) using PIL."""
    with Image.open(fname) as img:
        return img.size # (width, height)

def get_original_coords(kp_coords, img_orig_size, variation_info):
    """
    Transforms keypoint coordinates from variation space back to original image space.

    Args:
        kp_coords (np.ndarray): Keypoint coordinates [N, 2] in the variation space.
        img_orig_size (tuple): Original image size (width, height).
        variation_info (dict): Dictionary containing 'type' ('orig' or 'crop'),
                               'scale_factor' (scale used for resize),
                               'crop_box' ([x, y, w, h] in original coords, None if type is 'orig').

    Returns:
        np.ndarray: Keypoint coordinates [N, 2] in the original image space.
    """
    if len(kp_coords) == 0:
        return np.empty((0, 2))

    coords = kp_coords.copy() # Work on a copy

    # 1. Reverse scaling
    scale_factor = variation_info['scale_factor']
    coords /= scale_factor # Now coords are in the space of the original/cropped image (before resize)

    # 2. Reverse cropping offset
    if variation_info['type'] == 'crop' and variation_info['crop_box'] is not None:
        x_crop, y_crop, _, _ = variation_info['crop_box']
        coords[:, 0] += x_crop
        coords[:, 1] += y_crop

    # Ensure points are within original image bounds (optional, but good practice)
    # coords[:, 0] = np.clip(coords[:, 0], 0, img_orig_size[0] - 1)
    # coords[:, 1] = np.clip(coords[:, 1], 0, img_orig_size[1] - 1)

    return coords

In [6]:
def calculate_kornia_resize_scale(original_size_hw, target_resize):
    """
    Calculates the scale factor Kornia's default resize applies.
    Assumes aspect ratio is maintained and longer side is scaled to target_resize,
    only if longer side > target_resize.

    Args:
        original_size_hw (tuple): Original image size (H, W).
        target_resize (int): The target size for the longer side.

    Returns:
        float: The scale factor applied (processed_size / original_size).
    """
    h_orig, w_orig = original_size_hw
    max_orig_dim = max(h_orig, w_orig)

    if target_resize is None or target_resize <= 0 or max_orig_dim <= target_resize:
        # No resizing or scaling up is needed based on default logic
        return 1.0
    else:
        # Scale down the longer side to target_resize
        return target_resize / max_orig_dim

def transform_points_from_processed(kp_processed, processed_scale_factor, crop_box=None):
    """
    Transforms keypoint coordinates from the 'processed' scale space
    back to the original image space, accounting for scaling and cropping.

    Args:
        kp_processed (np.ndarray): Keypoint coordinates [N, 2] in the processed space (after scaling by ALIKED).
        processed_scale_factor (float): The calculated scale factor applied by ALIKED (processed_size / original_or_cropped_size).
        crop_box (list): [x, y, w, h] of the crop in original image coords, or None.

    Returns:
        np.ndarray: Keypoint coordinates [N, 2] in the original image space.
    """
    if len(kp_processed) == 0:
        return np.empty((0, 2), dtype=np.float32)

    coords = kp_processed.copy().astype(np.float32) # Ensure float32

    # 1. Reverse scaling (from processed scale back to original or cropped scale)
    if processed_scale_factor > 0:
        coords /= processed_scale_factor
    # else: scale_factor is 1.0, no change needed here

    # 2. Add cropping offset (from cropped coordinates back to original coordinates)
    if crop_box is not None and len(crop_box) == 4 and crop_box[2] > 0 and crop_box[3] > 0:
        x_crop, y_crop, _, _ = crop_box
        coords[:, 0] += x_crop
        coords[:, 1] += y_crop

    # Note: We don't clip to original bounds here, as that might discard valid points near edges.
    # Downstream steps should handle points outside bounds if necessary.

    return coords

# Remove the old get_keypoint_original_coords function entirely

In [7]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


# Must Use efficientnet global descriptor to get matching shortlists.
def get_global_desc(fnames, device = torch.device('cpu')):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_mac.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 30,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = get_global_desc(fnames, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

    # 只分析上三角（去掉对角线），避免重复
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]
    
    # 打印统计信息
    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
    print(f"USED 50%:  {np.percentile(dm_flat, 50):.4f}")
    print(f"75%:  {np.percentile(dm_flat, 75):.4f}")
    threshold = dm_flat.mean() + np.sqrt(3) * dm_flat.std()
    # removing half
    mask = dm <= np.percentile(dm_flat, 50)
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < threshold:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.1).eval().to(device, dtype)
    extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)

    # Calculate the expected scale factor ALIKED will apply
    # ALIKED uses preprocess_conf["resize"] on the *input image tensor*
    # Input image tensor size will be (H, W) after Kornia loading/conversion

    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            # Load original PIL image to get its size
            img_orig_pil = load_pil_image(img_path)
            if img_orig_pil is None:
                 print(f"Skipping ALIKED for {img_path}: PIL image loading failed.")
                 continue
            original_pil_size = img_orig_pil.size # (W, H)
            input_tensor_size_hw = (original_pil_size[1], original_pil_size[0]) # Convert (W, H) to (H, W)
            calculated_scale_factor = calculate_kornia_resize_scale(input_tensor_size_hw, resize_to)
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                kpts = transform_points_from_processed(
                                 kpts, calculated_scale_factor, crop_box=None)
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=20,verbose=True):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return



def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [8]:
import networkx as nx

import networkx as nx
from tqdm import tqdm

def match_with_lightglue_and_cluster(
    img_fnames,
    index_pairs,
    feature_dir='.featureout',
    device=torch.device('cpu'),
    min_matches=20,
    aliked_dis_min=100,
    verbose=True
):
    # 初始化 LightGlue 匹配器
    lg_matcher = KF.LightGlueMatcher(
        "aliked", {
            "width_confidence": -1,
            "depth_confidence": -1,
            "mp": 'cuda' in str(device)
        }
    ).eval().to(device)

    num_imgs = len(img_fnames)
    match_graph = nx.Graph()
    match_graph.add_nodes_from(range(num_imgs))

    # 加载关键点和描述子，准备写入匹配结果
    with h5py.File(os.path.join(feature_dir, 'keypoints.h5'), 'r') as f_kp, \
         h5py.File(os.path.join(feature_dir, 'descriptors.h5'), 'r') as f_desc, \
         h5py.File(os.path.join(feature_dir, 'matches.h5'), 'w') as f_match:

        # Step 1: 匹配每对图像
        for idx1, idx2 in tqdm(index_pairs, desc="LightGlue Matching"):
            img1 = os.path.basename(img_fnames[idx1])
            img2 = os.path.basename(img_fnames[idx2])

            kp1 = torch.from_numpy(f_kp[img1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[img2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[img1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[img2][...]).to(device)

            with torch.inference_mode():
                _, idxs = lg_matcher(
                    desc1, desc2,
                    KF.laf_from_center_scale_ori(kp1[None]),
                    KF.laf_from_center_scale_ori(kp2[None])
                )

            if len(idxs) == 0:
                continue

            n_matches = len(idxs)
            if verbose:
                print(f'{img1} - {img2}: {n_matches} matches')

            if n_matches >= min_matches:
                match_graph.add_edge(idx1, idx2, weight=n_matches)
                group = f_match.require_group(img1)
                group.create_dataset(img2, data=idxs.detach().cpu().numpy().reshape(-1, 2))

        # Step 2: 提取聚类（connected components）
        raw_clusters = list(nx.connected_components(match_graph))
        final_clusters = []
        outliers = set()

        for cluster in raw_clusters:
            cluster = list(cluster)
            subgraph = match_graph.subgraph(cluster)
            valid_nodes = []

            for node in cluster:
                strong_neighbors = [
                    neighbor for neighbor in subgraph.neighbors(node)
                    if subgraph[node][neighbor]['weight'] >= aliked_dis_min
                ]
                if strong_neighbors:
                    valid_nodes.append(node)
                else:
                    outliers.add(node)

            if len(valid_nodes) >= 2:
                final_clusters.append(valid_nodes)
            else:
                outliers.update(valid_nodes)

        # Step 3: 清理掉涉及 outlier 的匹配项（保留聚类内匹配）
        # outlier_names = {os.path.basename(img_fnames[i]) for i in outliers}

        # for group_name in f_match.keys():
        #     group = f_match[group_name]
        #     for dataset_name in list(group.keys()):
        #         if group_name in outlier_names or dataset_name in outlier_names:
        #             del group[dataset_name]
        #             if verbose:
        #                 print(f"Deleted match: {group_name} - {dataset_name} (outlier involved)")
        
        # Step 3: 清理掉涉及 outlier 的匹配项
        outlier_names = {os.path.basename(img_fnames[i]) for i in outliers}

        groups_to_delete = []

        for group_name in list(f_match.keys()):
            group = f_match[group_name]
            datasets_to_delete = []

            for dataset_name in list(group.keys()):
                # 删除与 outlier 有关的 match
                if group_name in outlier_names or dataset_name in outlier_names:
                    datasets_to_delete.append(dataset_name)
                    if verbose:
                        print(f"Deleted match: {group_name} - {dataset_name} (outlier involved)")

            # 先删除标记的 dataset
            for ds in datasets_to_delete:
                del group[ds]

            # 新增逻辑：如果剩下不到 3 个 match，就删掉整个 group
            if len(group.keys()) < 3:
                groups_to_delete.append(group_name)

        # 删除 group 本身
        for gname in groups_to_delete:
            del f_match[gname]
            if verbose:
                print(f"Deleted group: {gname} (too few matches)")

    if True:
        print(list(outliers))
        for i, cluster in enumerate(final_clusters):
            print(f"Cluster {i} ({len(cluster)} images):")
    return final_clusters, sorted(list(outliers))

In [9]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = False
data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "ETs" -> num_images=22
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "stairs" -> num_images=51


In [10]:
gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	'amy_gardens',
    	'ETs',
    	'fbk_vineyard',
    	'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []


print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue
    
    images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]
    if max_images is not None:
        images = images[:max_images]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:
        t = time()
        index_pairs = get_image_pairs_shortlist(
            images,
            sim_th = 0.5, # should be strict
            min_pairs = 10, # we should select at least min_pairs PER IMAGE with biggest similarity
            exhaustive_if_less = 20,
            device=device
        )
        timings['shortlisting'].append(time() - t)
        print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
        gc.collect()
    
        t = time()

        detect_aliked(images, feature_dir, 4096, device=device)
        gc.collect()
        timings['feature_detection'].append(time() - t)
        print(f'Features detected in {time() - t:.4f} sec')
        
        t = time()
        match_with_lightglue(images, index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        # match_with_lightglue_and_cluster(images, index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        timings['feature_matching'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')

        database_path = os.path.join(feature_dir, 'colmap.db')
        if os.path.isfile(database_path):
            os.remove(database_path)
        gc.collect()
        sleep(1)
        import_into_colmap(images_dir, feature_dir=feature_dir, database_path=database_path)
        output_path = f'{feature_dir}/colmap_rec_aliked'
        
        t = time()
        pycolmap.match_exhaustive(database_path)
        timings['RANSAC'].append(time() - t)
        print(f'Ran RANSAC in {time() - t:.4f} sec')
        
        # By default colmap does not generate a reconstruction if less than 10 images are registered.
        # Lower it to 3.
        mapper_options = pycolmap.IncrementalPipelineOptions()
        mapper_options.min_model_size = 8
        mapper_options.max_num_models = 25
        mapper_options.mapper.filter_max_reproj_error	 = 10.0

        os.makedirs(output_path, exist_ok=True)
        t = time()
        maps = pycolmap.incremental_mapping(
            database_path=database_path, 
            image_path=images_dir,
            output_path=output_path,
            options=mapper_options)
        sleep(1)
        timings['Reconstruction'].append(time() - t)
        print(f'Reconstruction done in  {time() - t:.4f} sec')
        print(maps)

        # clear_output(wait=False)
    
        registered = 0
        for map_index, cur_map in maps.items():
            for index, image in cur_map.images.items():
                prediction_index = filename_to_index[image.name]
                predictions[prediction_index].cluster_index = map_index
                predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
                predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
                registered += 1
        mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)
        gc.collect()
    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

Extracting on device cuda:0

Processing dataset "ETs": 22 images


100%|██████████| 22/22 [00:03<00:00,  6.56it/s]


Distance Matrix Statistics:
Min:  0.1504
Max:  0.4104
Mean: 0.2817
Std:  0.0495
20%:  0.2356
25%:  0.2547
USED 50%:  0.2804
75%:  0.3260
Shortlisting. Number of pairs to match: 142. Done in 8.7356 sec


100%|██████████| 22/22 [00:02<00:00,  9.60it/s]


Features detected in 2.6437 sec
Loaded LightGlue model


...

Features detected in 6.7174 sec
Loaded LightGlue model


100%|██████████| 650/650 [00:22<00:00, 28.76it/s]


Features matched in 22.7902 sec


 24%|██▍       | 296/1225 [00:00<00:00, 5582.42it/s]


Ran RANSAC in 7.7415 sec
Reconstruction done in  42.4735 sec
{0: Reconstruction(num_reg_images=17, num_cameras=17, num_points3D=1876, num_observations=5801), 1: Reconstruction(num_reg_images=9, num_cameras=9, num_points3D=606, num_observations=1961), 2: Reconstruction(num_reg_images=18, num_cameras=18, num_points3D=753, num_observations=2333)}
Dataset "stairs" -> Registered 44 / 51 images with 3 clusters

Results
Dataset "ETs" -> Registered 20 / 22 images with 1 clusters
Dataset "amy_gardens" -> Failed!
Dataset "fbk_vineyard" -> Failed!
Dataset "imc2023_haiper" -> Failed!
Dataset "imc2023_heritage" -> Failed!
Dataset "imc2023_theather_imc2024_church" -> Failed!
Dataset "imc2024_dioscuri_baalshamin" -> Failed!
Dataset "imc2024_lizard_pond" -> Failed!
Dataset "pt_brandenburg_british_buckingham" -> Failed!
Dataset "pt_piazzasanmarco_grandplace" -> Failed!
Dataset "pt_sacrecoeur_trevi_tajmahal" -> Failed!
Dataset "pt_stpeters_stpauls" -> Failed!
Dataset "stairs" -> Registered 44 / 51 image

In [11]:
# Must Create a submission file.

array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

image_id,dataset,scene,image,rotation_matrix,translation_vector
ETs_another_et_another_et001.png_public,ETs,cluster0,another_et_another_et001.png,0.661268918;-0.440250366;0.607373883;0.582461861;0.811561516;-0.045892112;-0.472717250;0.384119149;0.793089453,-1.575978893;-1.284916307;0.892816218
ETs_another_et_another_et002.png_public,ETs,cluster0,another_et_another_et002.png,0.668225169;-0.443927327;0.596995520;0.586943608;0.807661318;-0.056394998;-0.457134908;0.388087262;0.800259929,-1.454967468;-0.716854956;-0.038404077
ETs_another_et_another_et003.png_public,ETs,cluster0,another_et_another_et003.png,0.612977954;-0.458353214;0.643560689;0.634644128;0.770808755;-0.055503994;-0.470621779;0.442454737;0.763379949,-1.673201420;0.399983865;-1.206991713
ETs_another_et_another_et004.png_public,ETs,cluster0,another_et_another_et004.png,0.643454782;-0.437293645;0.628283544;0.525599679;0.849098629;0.052692464;-0.556516775;0.296320411;0.776197973,-1.571552970;-0.985321229;-0.919301776
ETs_another

In [12]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')